# 2.1 Advanced Python Constructs
## 2.1.1 Iterators, generator expressions and generators
### 2.1.1.1 Iterators
An iterator is an object adhering to the iterator protocol-basically this means that it has a next method, which, when called, returns the next item in the sequence, and when there's nothing to return, raises the **StopIteration** exception.
An iterator object allows to loop just once. It holds the state (position) of a single iteration, or from the other side, each loop over a sequence requires a single iterator object. This means that we can iterate over the same sequence more than once concurrently. Seperating the iteration logic from the sequence allows us to have more than one way of iteration.
Calling the __iter__ method on a containrer to create an iterator object is the most straightforward way to get hold of an iterator. The **iter** function does that for us, saving a few keystrokes.

In [1]:
nums = [1, 2, 3]
iter(nums)

In [2]:
nums.__iter__()

In [3]:
nums.__reversed__()

In [4]:
it = iter(nums)

In [5]:
next(it)

1

When used in a loop, StopIteration is swallowed and causes the loop to finish. But with explicit invocation, we can see that once the iterator is exhausted, accessing it raises an exception.

In [6]:
f = open('/etc/fstab')
f is f.__iter__()

True

### 2.1.1.2 Generator expressions
A second way in which iterator objects are created is through generator expressions, the basis for list comprehensions. To increase clarity, a generator expression must always be enclosed in parentheses or an expression. If round parentheses are used, then a generator iterator is created. If rectangular parentheses are used, the process is short-circuited and we get a list.

In [7]:
(i for i in nums)

<generator object <genexpr> at 0x7fc9c01dc910>

In [8]:
[i for i in nums]

[1, 2, 3]

In [9]:
list(i for i in nums)

[1, 2, 3]

In [10]:
# extends to dictionary and set comprehensions
{i for i in range(3)}

{0, 1, 2}

In [11]:
{i:i**2 for i in range(3)}

{0: 0, 1: 1, 2: 4}

### 2.1.1.3 Generators
A third way to create iterator objects is to call a generator function. A **generator** is a function containing the keyword yield. It must be noted that the mere presence of this keyword completely changes the nature of the function: this yield statement doesn't have to be invoked, or even reachable, but causes the function to be marked as a generator. When a normal function is called, the instructions contained in the body start to be executed. When a generator is called the execution stops before teh first instruction in the body. An invocation of a generator function creates a generator object, adhering to the iterator protocol. As with normal function invocations, concurrent and recursive invocations are allowed.
When next is called, the function is executed until the first yield. Each encountered **yield** statement gives a value becomes the return value of next. After executing the yield statement, teh execution of this function is suspended. 

In [12]:
def f():
    yield 1
    yield 2

In [13]:
f()

<generator object f at 0x7fc9c01dc6e0>

In [14]:
gen = f()
next(gen)

1

In [15]:
next(gen)

2

In [16]:
def f():
    print('-- start --')
    yield 3
    print('-- middle --')
    yield 4
    print('-- finished --')

In [17]:
gen = f()
next(gen)

-- start --


3

In [18]:
next(gen)

-- middle --


4

### 2.1.1.4 Bidirectional communication
Each yield statement causes a value to be passed to the caller. This is the reason for the introduction of generators by PEP 255. But communication in the reverse direction is also useful. One obvious way would be some external state, either a global variable or a shared mutable object. 
The first of the new methods is send(value), which is similar to **next()**, but passes value into the generator to be used for the value of the yield expression. In fact, g.next() and g.send(None) are equivalent.
The second of the new methods is throw(type, value=None, traceback=None) which is equivalent to:

In [30]:
import itertools
def g():
    print('-- start --')
    for i in itertools.count():
        print('-- yielding %i --' %i)
        try:
            ans = yield i
        except GeneratorExit:
            print('-- closing --')
            raise
        except Exception as e:
            print('-- yield raised %r --' %e)
        else:
            print('-- yield returned %s--' %ans)

In [31]:
it = g()
next(it)

-- closing --
-- yielding 4 --
-- start --
-- yielding 0 --


Exception RuntimeError: 'generator ignored GeneratorExit' in <generator object g at 0x7fc9c017ca50> ignored


0

In [32]:
it.send(11)

-- yield returned 11--
-- yielding 1 --


1

In [33]:
it.throw(IndexError)

-- yield raised IndexError() --
-- yielding 2 --


2

In [34]:
it.close()

-- closing --


### 2.1.1.5 Chaining generators
Let's say we are writing a generator and we want to yield a number of values generated by a second generator, a subgenerator. If yielding of values is the only concern, this can be performed without much difficulty using a loop such as:
```
subgen = some_other_generator()
for v in subgen:
    yield v
```

The **yield** statement has to be guarded by a try...except...finally structure similar to the one defined in the previous section to "debug" the generator function. It suffices to say that new syntax to properly yield from a subgenerator is being introduced in Python3.3:
```
yield from some_other_generator()
```

## 2.1.2 Decorators
Since functions and classes are objects, they can be passed around. Since they are mutable objects, they can be modified. The act of altering a function or class object after it has been constructed but before is bound to its name is called decorating.

There are two things hiding behind the name "decorator"--one is the function which does the work of decorating, i.e. performs the real woek, and the other one is the expression adhering to the decorator syntax, i.e. an at-symbol and the name of the decorating function.
Function can be decorated by using the decorator syntax for functions:
```
@decorator
def function():
    pass
```
* A function is defined in the standard way.
* An expression starting with @ placed before the function definition is the decorator. The part after @ must be a simple expression, usually this is just the name of a function or class. This part is evaluated first, and after the function defined below is ready, the decorator is called with the newly defined function object as the single argument. The value returned by the decorator is attached to the original name of the function.

The example above is quivalent to:
```
def function():
    pass
function = decoration(function)
```


### 2.1.2.1 Replacing or tweaking the original object
Decorators can either return the same function or class object or they can return a completely different object. In the first case, the decorator can exploit the fact that function and class objects are mutable and add attributes, e.g. add a docstring to a class.

### 2.1.2.2 Decorators implemented as classes and as functions
The only requirement on decorators is that they can be with a single argument. This means that decorators can be implemented as normal functions, or as clsses with a __call__ method, or in theory, even as lambda functions.

Let's compare the function and class approaches. The decorator expression(the part after @) can be either just a name, or a call. The bare-name approach is nice(less to type, looks cleaner, etc.), but is only possible when no arguments are needed to customise the decorator. Decorator written as functions can be used in those two cases:

In [37]:
def simple_decorator(function):
    print('doing decoration')
    return function

@simple_decorator
def function():
    print('inside function')


doing decoration


In [38]:
function()

inside function


In [41]:
def decorator_with_arguments(arg):
    print('defining the decorator')
    def _decorator(function):
        # in this inner function, arg is avaiable too
        print('doing decoration, %r' % arg)
        return function
    return _decorator

@decorator_with_arguments('abc')
def function():
    print('inside function')

defining the decorator
doing decoration, 'abc'


In [42]:
function()

inside function


The two trivial decorators above fall into the category of decorators which return the original function. If they were to return a new function, an extra level of nestedness would be required. In the worst case, three levels of nested functions.

In [44]:
def replacing_decorator_with_args(arg):
    print('defining the decorator')
    def _decorator(function):
        # in this inner function, arg is available too
        print('doing decorator, %r' %arg)
        def _wrapper(*args, **kwargs):
            print('inside wrapper, %r %r' %(args, kwargs))
            return function(*args, **kwargs)
        return _wrapper
    return _decorator

@replacing_decorator_with_args('abc')
def function(*args, **kwargs):
    print('inside function, %r %r' %(args, kwargs))
    return 14

defining the decorator
doing decorator, 'abc'


In [45]:
function(11, 12)

inside wrapper, (11, 12) {}
inside function, (11, 12) {}


14

The _wrapper function is defined to accept all positional and keyword arguments. In general we cannot know what arguments the decorated function is supposed to accept, so the wrapper function just passes everything to the wrapped function. One unfortunate consequence is that the apparent argument list is misleading.

Compared to decorators defined as functions, complex decorators defined as classes are simpler. When an object is created, the **\_\_init\_\_** method is only allowed to return None, and the type of the created object cannot be changed. This means that when a decorator is defined as a class, it doesn't make much sense to use the argument-less form: the final decorated object would just be an instance of the decorating class, returned by the constructor call, which is not very usefull. Therefore it's enough to discuss class-based decorators where arguments are given in the decorator expression and the decorator **\_\_init\_\_** method is used for decorator construction. 


In [48]:
class decorator_class(object):
    def __init__(self, arg):
        # this method is called in the decorator expression
        print('in decorator init, %s' % arg)
        self.arg = arg
    def __call__(self, function):
        # this method is called to do the job
        print('in decorator call, %s' % self.arg)
        return function
deco_instance = decorator_class('foo')

in decorator init, foo


In [50]:
@deco_instance
def function(*args, **kwargs):
    print('in function, %s %s' %(args, kwargs))

in decorator call, foo


In [51]:
function()

in function, () {}


In [53]:
class replacing_decorator_class(object):
    def __init__(self, arg):
        # this method is called in the decorator expression
        print('in decorator init, %s' % arg)
        self.arg = arg
    def __call__(self, function):
        # this method is called to do the job
        print('in decorator call, %s' % self.arg)
        self.function = function
        return self._wrapper
    def _wrapper(self, *args, **kwargs):
        print('in the wrapper, %s %s' %(args, kwargs))
        return self.function(*args, **kwargs)

In [54]:
deco_instance = replacing_decorator_class('foo')

in decorator init, foo


In [58]:
@deco_instance
def function(*args, **kwargs):
    print('in function, %s %s'%(args, kwargs))
    

in decorator call, foo


In [59]:
function(11, 12)

in the wrapper, (11, 12) {}
in function, (11, 12) {}


### 2.1.2.3 Copying the docstring and other attributes of the original function
When a new function is returned by the decorator to replace the originalfunction, an unfoutunate consequence is that the original function name, the original docstring, the original argument list are lost. Those attributes of teh original function can partially be 'transplanted' to the new function by setting **\_\_doc\_\_**(the docstring), **\_\_module\_\_** and **\_\_name\_\_** (the full name of the function), and **\_\_annotations\_\_** (extra information about arguments and the return value of the function available in Python 3). This can be done automatically by using **functools.update_wrapper**.

```
functools.update_wrapper(wrapper, wrapped)
```
"Update a wrapper function to look like the wrapped function."

In [61]:
import functools
def replacing_decorator_with_args(arg):
    print('defining the decorator')
    def _decorator(function):
        print('doing decoration, %r' % arg)
        def _wrapper(*arg, **kwargs):
            print('inside wrapper, %r %r' %(args, kwargs))
            return function(*args, **kwargs)
        return functools.update_wrapper(_wrapper, function)
    return _decorator

@replacing_decorator_with_args('abc')
def function():
    "extensive documentation"
    print('inside function')
    return 14

defining the decorator
doing decoration, 'abc'


One important thing is missing from the list of attributes which can be copied to the replacement function: the argument list. The default values for arguments can be modified through the **\_\_defaults\_\_**, **\_\_kwdefaults\_\_** attributes,  but unfortunately the argument list which will be confusing for the user of the function, but unfortunately the argument list itself cannot be set as an attribute.

### 2.1.2.4 Examples in the standard library
First, it should be mentioned that there's a number of useful decorators available in the standard library. There are three decorators which really form a part of the language:
* **classmethod** causes a method to become a "class method", which means that it can be invoked without creating an instance of the class. When a normal method is invoked, the interpreter inserts the instance object as the first positional parameter, `self`. When a class method is invoked, the class itself is given as the first parameter, often called `cls`.
Class methods are still accessible through the class' namespace, so they don't pollute teh module's namespace. Class methods can be used to provide alternative constructors:

In [62]:
class Array(object):
    def __init__(self, data):
        self.data = data
    
    @classmethod
    def fromfile(cls, file):
        data = numpy.load(file)
        return cls(data)

This is cleaner then using a multitude of flags to **\_\_init\_\_**.
* **staticmethod** is applied to methods to make them "static", i.e. basically a normal funtion, but accessible through the class namespace. This can be useful when the function is only needed inside this class(its name would then be prefixed with _), or when we want the user to think of the method as connected to the class, despite an implementation which doesn't require this.
* **property** is the pythonic answer to the problem of `getters` and setters. A method decorated with property becomes a getter which is automatically called on attribute access.

In [64]:
class A(object):
    @property
    def a(self):
        "an important attribute"
        return "a value"

A.a

In [65]:
A().a

'a value'

In this example, A.a is an read-only attribute. It is also documented: help(A) includes the docstring for attribute a taken from the getter method. 